In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
single = pd.read_csv(r'C:\Users\James Zhang\Desktop\sorrentum\sorrentum_sandbox\projects\SorrIssue2_Cross_exchange_arbitrage_CEX_CEX\single_merged.csv')

In [3]:
single = single.set_index('timestamp')

### quickly convert it to a multiindex df

In [5]:
def convert_to_multi(df, timestamp):
    # creating most inner column 
    cols_str = ""
    cols = list(df.columns)
    for elem in cols:
        cols_str = "".join([cols_str, elem]) + " "
    len(cols_str.split())

    # creating the middle column (the exchange: okx, binance_spot, binance_future, binanceus)
    exchanges_str = ""
    for col in cols:
        h, s = col.rfind('-') + 1, col.rfind(':')
        exchanges_str += col[h:s] + " "

    # creating outer column (the feature: volume or vwap)
    features_str = "volume " * int(len(cols_str.split())/2) + "vwap " * int(len(cols_str.split())/2)

    new_df = pd.DataFrame(np.array(df), columns=[features_str.split(), exchanges_str.split(), cols_str.split()]) # this line needs to include a new level
    # new_df.set_index(timestamp, inplace=True)

    d = dict(zip(new_df.columns.levels[2], [word[2][word[2].rfind(':') + 1:] for word in new_df.columns]))
    new_df = new_df.rename(columns=d, level=2)
    new_df = new_df.set_index(timestamp['timestamp'].astype('datetime64[ms]'))
    return new_df

In [6]:
df = convert_to_multi(single, pd.DataFrame(single.index))

In [19]:
df.head()

volume                                        \
                    binance_futures                                         
                           APE_USDT AVAX_USDT AXS_USDT BAKE_USDT BNB_USDT   
timestamp                                                                   
2019-09-08 17:57:00             NaN       NaN      NaN       NaN      NaN   
2019-09-08 17:58:00             NaN       NaN      NaN       NaN      NaN   
2019-09-08 17:59:00             NaN       NaN      NaN       NaN      NaN   
2019-09-08 18:00:00             NaN       NaN      NaN       NaN      NaN   
2019-09-08 18:01:00             NaN       NaN      NaN       NaN      NaN   

                                                                   ...  \
                                                                   ...   
                    BTC_BUSD BTC_USDT CRV_USDT CTK_USDT DOGE_USDT  ...   
timestamp                                                          ...   
2019-09-08 17:57:00      NaN    0.001      NaN      NaN       NaN  ...   
2019-09-08 17:58:00      NaN    0.000      NaN      NaN       NaN  ...   
2019-09-08 17:59:00      NaN    0.001      NaN      NaN       NaN  ...   
2019-09-08 18:00:00      NaN    0.000      NaN      NaN       NaN  ...   
2019-09-08 18:01:00      NaN    0.000      NaN      NaN       NaN  ...   

                        vwap                                          \
                         okx                                           
                    FTM_USDT GMT_USDT LINK_USDT MATIC_USDT NEAR_USDT   
timestamp                                                              
2019-09-08 17:57:00      NaN      NaN       NaN        NaN       NaN   
2019-09-08 17:58:00      NaN      NaN       NaN        NaN       NaN   
2019-09-08 17:59:00      NaN      NaN       NaN        NaN       NaN   
2019-09-08 18:00:00      NaN      NaN       NaN        NaN       NaN   
2019-09-08 18:01:00      NaN      NaN       NaN        NaN       NaN   

                                                                       
                                                                       
                    SAND_USDT SOL_USDT STORJ_USDT WAVES_USDT XRP_USDT  
timestamp                                                              
2019-09-08 17:57:00       NaN      NaN        NaN        NaN      NaN  
2019-09-08 17:58:00       NaN      NaN        NaN        NaN      NaN  
2019-09-08 17:59:00       NaN      NaN        NaN        NaN      NaN  
2019-09-08 18:00:00       NaN      NaN        NaN        NaN      NaN  
2019-09-08 18:01:00       NaN      NaN        NaN        NaN      NaN  

[5 rows x 180 columns]

### the above is the correct multiindex df

# next task: instantaneous arbitrage approximation
we will first assume no transaction costs
idea:

- for each coin, at any time (we can resample later), calculate maximum price dislocation across any exchange
    - buy lower one and sell higher one at the lower corresponding volume
- sum them all at the end

In [31]:
coins = list(df.columns.levels[2])
exchanges = list(df.columns.levels[1])
exchanges

['binance_futures', 'binance_spot', 'binanceus', 'okx']

In [20]:
temp = df.swaplevel(1, 2, 1).sort_index(axis=1)
df2 = temp.swaplevel(0, 1, 1).sort_index(axis=1)
df2

APE_USDT                             \
                             volume                              
                    binance_futures binance_spot binanceus okx   
timestamp                                                        
2019-09-08 17:57:00             NaN          NaN       NaN NaN   
2019-09-08 17:58:00             NaN          NaN       NaN NaN   
2019-09-08 17:59:00             NaN          NaN       NaN NaN   
2019-09-08 18:00:00             NaN          NaN       NaN NaN   
2019-09-08 18:01:00             NaN          NaN       NaN NaN   
...                             ...          ...       ...  ..   
2023-01-31 23:55:00             NaN       154.52       0.0 NaN   
2023-01-31 23:56:00             NaN       264.84       0.0 NaN   
2023-01-31 23:57:00             NaN       287.58       0.0 NaN   
2023-01-31 23:58:00             NaN       887.80       0.0 NaN   
2023-01-31 23:59:00             NaN       404.81       0.0 NaN   

                                                                \
                               vwap                              
                    binance_futures binance_spot binanceus okx   
timestamp                                                        
2019-09-08 17:57:00             NaN          NaN       NaN NaN   
2019-09-08 17:58:00             NaN          NaN       NaN NaN   
2019-09-08 17:59:00             NaN          NaN       NaN NaN   
2019-09-08 18:00:00             NaN          NaN       NaN NaN   
2019-09-08 18:01:00             NaN          NaN       NaN NaN   
...                             ...          ...       ...  ..   
2023-01-31 23:55:00             NaN     9.411818   9.78506 NaN   
2023-01-31 23:56:00             NaN     9.411818   9.78506 NaN   
2023-01-31 23:57:00             NaN     9.411818   9.78506 NaN   
2023-01-31 23:58:00             NaN     9.411818   9.78506 NaN   
2023-01-31 23:59:00             NaN     9.411817   9.78506 NaN   

                          AVAX_USDT               ...   WAVES_USDT      \
                             volume               ...         vwap       
                    binance_futures binance_spot  ... binance_spot okx   
timestamp                                         ...                    
2019-09-08 17:57:00             NaN          NaN  ...          NaN NaN   
2019-09-08 17:58:00             NaN          NaN  ...          NaN NaN   
2019-09-08 17:59:00             NaN          NaN  ...          NaN NaN   
2019-09-08 18:00:00             NaN          NaN  ...          NaN NaN   
2019-09-08 18:01:00             NaN          NaN  ...          NaN NaN   
...                             ...          ...  ...          ...  ..   
2023-01-31 23:55:00             NaN       599.82  ...    10.713475 NaN   
2023-01-31 23:56:00             NaN       584.41  ...    10.713472 NaN   
2023-01-31 23:57:00             NaN      1333.10  ...    10.713472 NaN   
2023-01-31 23:58:00             NaN       642.07  ...    10.713444 NaN   
2023-01-31 23:59:00             NaN       304.73  ...    10.713440 NaN   

                           XRP_USDT                             \
                             volume                              
                    binance_futures binance_spot binanceus okx   
timestamp                                                        
2019-09-08 17:57:00             NaN          NaN       NaN NaN   
2019-09-08 17:58:00             NaN          NaN       NaN NaN   
2019-09-08 17:59:00             NaN          NaN       NaN NaN   
2019-09-08 18:00:00             NaN          NaN       NaN NaN   
2019-09-08 18:01:00             NaN          NaN       NaN NaN   
...                             ...          ...       ...  ..   
2023-01-31 23:55:00             NaN     135505.0   11914.6 NaN   
2023-01-31 23:56:00             NaN     177663.0   11914.6 NaN   
2023-01-31 23:57:00             NaN     197800.0   11914.6 NaN   
2023-01-31 23:58:00             NaN     159875.0   11914.6 NaN   
2023-01-31

# note that we can easily swap levels of the multiindex columns
df2 has the following
- outer column: the security (27 coins on different exchanges)
- middle column: the feature (volume or vwap)
- inner column: the exchange (binance_futures, binance_spots, binanceus, okx)

In [17]:
df2

APE_USDT                             \
                             volume                              
                    binance_futures binance_spot binanceus okx   
timestamp                                                        
2019-09-08 17:57:00             NaN          NaN       NaN NaN   
2019-09-08 17:58:00             NaN          NaN       NaN NaN   
2019-09-08 17:59:00             NaN          NaN       NaN NaN   
2019-09-08 18:00:00             NaN          NaN       NaN NaN   
2019-09-08 18:01:00             NaN          NaN       NaN NaN   
...                             ...          ...       ...  ..   
2023-01-31 23:55:00             NaN       154.52       0.0 NaN   
2023-01-31 23:56:00             NaN       264.84       0.0 NaN   
2023-01-31 23:57:00             NaN       287.58       0.0 NaN   
2023-01-31 23:58:00             NaN       887.80       0.0 NaN   
2023-01-31 23:59:00             NaN       404.81       0.0 NaN   

                                                                \
                               vwap                              
                    binance_futures binance_spot binanceus okx   
timestamp                                                        
2019-09-08 17:57:00             NaN          NaN       NaN NaN   
2019-09-08 17:58:00             NaN          NaN       NaN NaN   
2019-09-08 17:59:00             NaN          NaN       NaN NaN   
2019-09-08 18:00:00             NaN          NaN       NaN NaN   
2019-09-08 18:01:00             NaN          NaN       NaN NaN   
...                             ...          ...       ...  ..   
2023-01-31 23:55:00             NaN     9.411818   9.78506 NaN   
2023-01-31 23:56:00             NaN     9.411818   9.78506 NaN   
2023-01-31 23:57:00             NaN     9.411818   9.78506 NaN   
2023-01-31 23:58:00             NaN     9.411818   9.78506 NaN   
2023-01-31 23:59:00             NaN     9.411817   9.78506 NaN   

                          AVAX_USDT               ...   WAVES_USDT      \
                             volume               ...         vwap       
                    binance_futures binance_spot  ... binance_spot okx   
timestamp                                         ...                    
2019-09-08 17:57:00             NaN          NaN  ...          NaN NaN   
2019-09-08 17:58:00             NaN          NaN  ...          NaN NaN   
2019-09-08 17:59:00             NaN          NaN  ...          NaN NaN   
2019-09-08 18:00:00             NaN          NaN  ...          NaN NaN   
2019-09-08 18:01:00             NaN          NaN  ...          NaN NaN   
...                             ...          ...  ...          ...  ..   
2023-01-31 23:55:00             NaN       599.82  ...    10.713475 NaN   
2023-01-31 23:56:00             NaN       584.41  ...    10.713472 NaN   
2023-01-31 23:57:00             NaN      1333.10  ...    10.713472 NaN   
2023-01-31 23:58:00             NaN       642.07  ...    10.713444 NaN   
2023-01-31 23:59:00             NaN       304.73  ...    10.713440 NaN   

                           XRP_USDT                             \
                             volume                              
                    binance_futures binance_spot binanceus okx   
timestamp                                                        
2019-09-08 17:57:00             NaN          NaN       NaN NaN   
2019-09-08 17:58:00             NaN          NaN       NaN NaN   
2019-09-08 17:59:00             NaN          NaN       NaN NaN   
2019-09-08 18:00:00             NaN          NaN       NaN NaN   
2019-09-08 18:01:00             NaN          NaN       NaN NaN   
...                             ...          ...       ...  ..   
2023-01-31 23:55:00             NaN     135505.0   11914.6 NaN   
2023-01-31 23:56:00             NaN     177663.0   11914.6 NaN   
2023-01-31 23:57:00             NaN     197800.0   11914.6 NaN   
2023-01-31 23:58:00             NaN     159875.0   11914.6 NaN   
2023-01-31

let's make this algorithm for 1 coin for now, say Bitcoin. let us denote $\mu =$ arbitrage index.

$$\mu = \min(volume[i], volume[j]) * (price_j - price_i)$$

arbitrage index df

                  price_i        price_j          i                j              volume[i]     volume[j]       arbitrage_index
        9:30        10              8         binanceus     binance_futures           2            5              
        9:31        12              8            okx        binance_futures           3            7

where $i, j$ are the same coin but on different exchanges. they represent different indices in the df.

Let coin $price_i$ be the minimum price of some arbitrary coin and let $price_j$ be the maximum price of the same arbirary coin at some time $t$

We can separate the min volume column into two columns volume_j, volume_i -> this works too

In [47]:
df2['BTC_USDT']

volume                             \
                    binance_futures binance_spot binanceus okx   
timestamp                                                        
2019-09-08 17:57:00           0.001          NaN       NaN NaN   
2019-09-08 17:58:00           0.000          NaN       NaN NaN   
2019-09-08 17:59:00           0.001          NaN       NaN NaN   
2019-09-08 18:00:00           0.000          NaN       NaN NaN   
2019-09-08 18:01:00           0.000          NaN       NaN NaN   
...                             ...          ...       ...  ..   
2023-01-31 23:55:00             NaN    132.98878  0.665210 NaN   
2023-01-31 23:56:00             NaN    186.18800  2.498794 NaN   
2023-01-31 23:57:00             NaN    110.49868  1.791509 NaN   
2023-01-31 23:58:00             NaN    163.33895  1.062466 NaN   
2023-01-31 23:59:00             NaN    134.69127  0.243654 NaN   

                               vwap                                  
                    binance_futures  binance_spot     binanceus okx  
timestamp                                                            
2019-09-08 17:57:00         10000.0           NaN           NaN NaN  
2019-09-08 17:58:00         10000.0           NaN           NaN NaN  
2019-09-08 17:59:00         10000.0           NaN           NaN NaN  
2019-09-08 18:00:00         10000.0           NaN           NaN NaN  
2019-09-08 18:01:00         10000.0           NaN           NaN NaN  
...                             ...           ...           ...  ..  
2023-01-31 23:55:00             NaN  28969.897150  27207.485001 NaN  
2023-01-31 23:56:00             NaN  28969.884672  27207.471198 NaN  
2023-01-31 23:57:00             NaN  28969.877264  27207.461297 NaN  
2023-01-31 23:58:00             NaN  28969.866312  27207.455428 NaN  
2023-01-31 23:59:00             NaN  28969.857268  27207.454079 NaN  

[1787402 rows x 8 columns]

using this method we can easily get the min and max price as well as the exchange that have these min and max prices

In [ ]:
df2['BTC_USDT']['vwap'][exchanges].max(axis=1)

In [ ]:
df2['BTC_USDT']['vwap'][exchanges].idxmax(axis=1)